# AutoRec cs3639 Recommendation Systems course IDC

### here will be general explanations

In [13]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## datasets

In this project, we will use 2 datasets:
* **movielens**, which can be downloaded using `utils.datasets_download.py` or straight from [here](http://files.grouplens.org/datasets/movielens/).
* **netflixprize**, which can be downloaded from this [semi-parsed version from kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data) or from this [raw version](https://archive.org/download/nf_prize_dataset.tar)

**NOTE**: for the notebook to run properly, you should save you dataset under `data` folder and `movielens` folder for the movielens dataset and `netflix` folder for the netflixprize dataset.
i.e `data/movielens` folder and `data/netflix` folder respectively.

In [53]:
from src.data_prep import movielens_load
train, test = movielens_load(1)
print(train.shape)
train

(80000, 4)


,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [54]:
# TODO:
# - Add dataloder class
# - Add batch size
# - Add bias for the MatrixFactorization

In [55]:
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, k=15):
        super().__init__()
        self.k = k
        self.num_users = num_users
        self.num_items = num_items

        # create the latent matrixs
        self.users_emb = nn.Embedding(num_users, k)
        self.items_emb = nn.Embedding(num_items, k)

        self.batch_size = 100


    def forward(self, user, item):
        user = self.users_emb(user)
        item = self.items_emb(item)
        return (user*item).sum(axis=1)



In [56]:
# TODO:
# - add function that will calculate the validation loss

In [85]:
def train_epocs(train, model, epochs=10, lr=0.001, reg=0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=reg)
    for i in range(epochs):
        model.train()
        # users and items indexes start 1, therefore we use the -1
        users = torch.LongTensor(train.user_id.values-1).to(device)
        items = torch.LongTensor(train.item_id.values-1).to(device)
        ratings = torch.FloatTensor(train.rating.values).to(device) # rating is our label

        preds = model(users, items)
        loss = torch.sqrt(nn.functional.mse_loss(preds, ratings))

        # backpropagation
        optimizer.zero_grad()
        loss.backward()

        # update
        optimizer.step()

        print(f'train RMSE: {loss.item()}')


In [86]:
num_users = train.user_id.max()
num_items = train.item_id.max()
print(num_users, num_items)
model = MatrixFactorization(num_users, num_items).to(device)


943 1682


In [87]:
%%time
train_epocs(train, model, epochs=1000, lr=0.001, reg=0.001)

train RMSE: 5.372889995574951
train RMSE: 5.368236541748047
train RMSE: 5.36359167098999
train RMSE: 5.358957290649414
train RMSE: 5.354332447052002
train RMSE: 5.349717140197754
train RMSE: 5.345112323760986
train RMSE: 5.340517044067383
train RMSE: 5.335933208465576
train RMSE: 5.331358909606934
train RMSE: 5.32679557800293
train RMSE: 5.322242259979248
train RMSE: 5.317700386047363
train RMSE: 5.313168525695801
train RMSE: 5.308647632598877
train RMSE: 5.304137706756592
train RMSE: 5.299638271331787
train RMSE: 5.295149803161621
train RMSE: 5.290672779083252
train RMSE: 5.286205768585205
train RMSE: 5.281750202178955
train RMSE: 5.277305603027344
train RMSE: 5.2728729248046875
train RMSE: 5.268450736999512
train RMSE: 5.264039516448975
train RMSE: 5.259639739990234
train RMSE: 5.255250930786133
train RMSE: 5.250873565673828
train RMSE: 5.24650764465332
train RMSE: 5.242153167724609
train RMSE: 5.237809658050537
train RMSE: 5.233477592468262
train RMSE: 5.229157447814941
train RMSE: 

In [ ]:
class AutoRec(nn.Module):
    """
    AutoRec model
    """
    def __init__(self, ):
        super().__init__()
        self.encoder = nn.Dense()